<a href="https://colab.research.google.com/github/archietech-ai/tokenizer/blob/main/Masked_Language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎭 **Masked Language Modeling with BERT**

Explore masked language modeling (MLM) using the BERT model to understand context and predict missing words in sentences.

## 🛠️ Setup and Installation

Begin by installing the necessary libraries to manage data processing and modeling.

In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


## 📚 Importing Libraries

Import essential modules for our tasks.

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer  # AutoModelForMaskedLM is a generic class for loading pre-trained masked language models. It automatically selects the appropriate model architecture based on the name or path of the pre-trained model you provide.
import pandas as pd
import numpy as np
from scipy.special import softmax

## 🤖 Model Setup

Load the pre-trained BERT model and tokenizer, specifically designed for masked language modeling.

In [3]:
model_name = "bert-base-cased"

# Loading the pre-trained model and tokenizer
model = AutoModelForMaskedLM.from_pretrained(model_name) #  loads the correct model with a masked language modeling head
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

## 🎭 Defining the Mask Token

Identify the mask token used by BERT to signify where predictions are needed in the sentence.

## ✏️ Creating the Input Sentence

Craft a sentence with a missing word indicated by the mask token, to test the model's predictive power.

In [5]:
# Defining the mask token
mask = tokenizer.mask_token # Retrieves the special "mask token" used by the tokenizer.The mask_token is a special token added to the tokenizer's vocabulary during pre-training.

# Defining the sentence
sentence = f"I want to {mask} pizza for tonight."

# Tokenizing the sentence
tokens = tokenizer.tokenize(sentence)

## 🔍 Tokenization and Encoding

Tokenize and encode the sentence to format it properly for the model.

## 📈 Model Prediction

Feed the encoded inputs to the model and extract logits for predictions.

In [12]:
# Encoding the input sentence and getting model predictions
encoded_inputs = tokenizer(sentence, return_tensors="pt")
output = model(**encoded_inputs)
'''
The output of the above ine is an object that has the following features
MaskedLMOutput(
    loss=None,
    logits=tensor([...]),
    hidden_states=None,
    attentions=None
)

See the next cell

loss=None: --> loss is None because no labels were provided to the model during the forward pass.
If labels were passed, the model would calculate the MLM loss (cross-entropy loss).

logits: --> The logits tensor has shape (batch_size, sequence_length, vocab_size).
These are the raw prediction scores (logits) for each token position and each vocabulary word.
'''

'\nMaskedLMOutput(\n    loss=None, \n    logits=tensor([...]), \n    hidden_states=None, \n    attentions=None\n)\n'

In [10]:
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3723,  -7.2489,  -7.4421,  ...,  -6.3119,  -5.9369,  -6.4257],
         [ -7.9311,  -8.2282,  -8.0326,  ...,  -6.7387,  -6.4877,  -6.9525],
         [-12.0500, -11.7972, -12.5776,  ...,  -8.4518,  -6.7310,  -8.2586],
         ...,
         [-10.2204, -10.4315,  -9.9993,  ...,  -7.9570,  -6.7194,  -9.3618],
         [-12.4471, -12.5367, -12.5614,  ...,  -9.9086,  -9.4219, -11.1770],
         [-14.3657, -14.5227, -15.0017,  ..., -11.9715, -11.6569, -13.4498]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
# Detaching the logits from the model output and converting to numpy array
logits = output.logits.detach().numpy()[0]


## 🔎 Analyzing Predictions

Retrieve logits for the masked token and calculate confidence scores for possible replacements.

In [7]:
# Extracting the logits for the masked token and calculating the confidence scores
masked_logits = logits[tokens.index(mask) + 1]
confidence_scores = softmax(masked_logits)

## 📝 Displaying Top Predictions

Cycle through the top 5 predicted tokens, substituting the masked token in the original sentence to show the model's suggestions.


In [8]:
# Iterating over the top 5 predicted tokens and printing the sentences with the masked token replaced
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # print(pred_token, score)
    print(sentence.replace(mask, pred_token))

I want to have pizza for tonight.
I want to get pizza for tonight.
I want to eat pizza for tonight.
I want to make pizza for tonight.
I want to order pizza for tonight.
